# Scraping 
- In this Project multiple scraping scripts are used for different band websites as base for the later dataset.

## MetalScraper 
- Main script 
- The goal of Metalscraper.py is to fetch genre, Bandname and Link for the band-logo font pictures from each band-site. 
- scraping of the HTML Tree structure is necessary.   
## LinkScraper
- The Linkscraper is used for scraping a list of links for the dataset from the websites. Those links stand for all the bands which all have a single page on the website. So the goal has to be, to get allthe links in order to fetch all metadata from the band-sites later. 


# Libraries and packages
- beautiful soup 
- requests 
- urllib
- For all libs. and packages see: requirement.txt ! 

In [7]:
import urllib
from bs4 import BeautifulSoup
import requests

## Linkscraper script 

In [8]:
import requests
varlist = open("List.csv", "w")
letterlist = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "NBR"]

varlist: list for only writing-output in Csv-file "List.csv" -> output= all Links for all bands to use with the metal scraper script in the next step

letterlist: List for all Bandnames categorized by start-letter of their Name on the website 

In [ ]:
for letter in letterlist:

    counter = 200
    response = requests.get(f"https://www.metal-archives.com/label/ajax-list/json/1/l/{letter}?sEcho=1&iColumns=7&sColumns=&iDisplayStart=0&iDisplayLength={counter}&mDataProp_0=0&mDataProp_1=1&mDataProp_2=2&mDataProp_3=3&mDataProp_4=4&mDataProp_5=5&mDataProp_6=6&iSortCol_0=1&sSortDir_0=asc&iSortingCols=1&bSortable_0=false&bSortable_1=true&bSortable_2=true&bSortable_3=true&bSortable_4=true&bSortable_5=false&bSortable_6=true&_=1708260299235", headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})
    response_json = response.json()
    counter = response_json["iTotalRecords"]
    print (counter, letter)

    response = requests.get(f"https://www.metal-archives.com/label/ajax-list/json/1/l/{letter}?sEcho=1&iColumns=7&sColumns=&iDisplayStart=0&iDisplayLength={counter}&mDataProp_0=0&mDataProp_1=1&mDataProp_2=2&mDataProp_3=3&mDataProp_4=4&mDataProp_5=5&mDataProp_6=6&iSortCol_0=1&sSortDir_0=asc&iSortingCols=1&bSortable_0=false&bSortable_1=true&bSortable_2=true&bSortable_3=true&bSortable_4=true&bSortable_5=false&bSortable_6=true&_=1708260299235", headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})
    response_json = response.json()
    links = response_json["aaData"]       
    output_String = letter

    for entry in links:
        output_String += ";" + entry[1].split('"')[1]
    varlist.write(output_String + "\n")

This website request is used for getting the starting-letter and a counter of how many bands fit withina certain letter-category.   
(counter 200 = default for website requests)
The f string in the request is used for changing parameters to the variables "letter" and "count" instead of "given letter" and "given count". 
As first response we get a json-object for total records per letter-category. 

The second request is used to get all total records per letter, with links as output of all http-links for all of the Bands from the request. 
As output string we get  an empty string but with entry for first letter. 

The complete output is made by the output string and the links in csv-format into List.csv.

## Linklist converter
- coverts csv to txt fro working with linklist in metal scraper script

In [ ]:
CompleteLinkList = open("Linklist.txt", "w")
LinkList = open("List.csv")

for line in LinkList:
    line = line.rstrip("\n")
    lineArray = line.split(";")[1:]
    for entry in lineArray:
        if entry != "":
            CompleteLinkList.write(entry + "\n")

## MetalScraper Script

In [1]:
import urllib
from bs4 import BeautifulSoup
import requests

Linklist = open("Linklist.txt")                                                             #opens Linklist 
Datalist = open("Datalist.csv", "a", encoding="utf-8")                                      #opens Datalist with all scraped link, as "write only", uses utf8-encoding to encode special characters, unicode was used before but is not sufficent

In [ ]:

for line in Linklist:                                                                                        #for every link from linklist:
    page = requests.get(line, headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})           #get request ato website for the link

    if page.status_code == 200:                                                                                 #200 status = go for request from website (OK)
        content = page.content                                                                                  

    
    DOMdocument = BeautifulSoup(content, 'html.parser')                                                         #parse website content with bs4

    
    name = DOMdocument.find("h1", class_='band_name')                                                           #search for name in html doc
    
    logo_prior = DOMdocument.find("a",  id="logo")                                                              #search for logo in html doc / gives 'None' back, if no entry found
    if logo_prior  != None:                                                                                     #if logo entry found
        logo = logo_prior.attrs['href']                                                                         #get link from href
    else:
        print(f"{name} has no logo!")                                                                           #else: print out that the band has no logo
        continue                                                                                                #jump over all bands without logo entry -> jumps to start of loop and continue 
  

    Genre = DOMdocument.find_all(['dl','dd'], {'class': "float_right"})                                         # find genre in html doc -> marked by d1 & dd -> float right = pagearchitecture frontend (genre on the rioght of the page) 
    for div in Genre:                                                                                           #find_all returns iterable but only with 1 entry 
        genre = div.text.split("\n")[2]                                                                         #takes entry with genre and gives back textliefert enclosed by \n



    print(name.get_text())
    print(genre)
    print(logo)
    Datalist.write(name.get_text()+";"+genre+";"+logo+"\n")




The output is Datalist.csv that now holds every band that has a logo wtih the attributes: bandname, genre and logo link 

## Picture download
- to download the pictures from the logolinks a new script is needed 
- This script provides the download for all pictures and opens up a new folder for every new genre, that is spotted 

In [ ]:
import os, requests, urllib3
import time
import re

                                                                                                                                                #read Datalist.csv 
                                                                                                                                                #{'Connection': 'keep-alive', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'User-Agent': 'python-requests/2.13.0'}

Datalist = open("Datalist.csv", encoding="utf-8")                                                                                               #loop für all entries in Datalist
for entry in Datalist:                                                                                                                          
                                                                                                                                                #parse entries for ";" 
    entryArr = entry.split(";")                                                                                                                 #Array comes back -> Name Genre Link
    bandName = entryArr[0]
    genre = entryArr[1]
    logo = entryArr[2].rstrip("\n")                                                                                                             # split Genre for"/"" 
    genreArr = genre.split("/")

    for genre in genreArr:                                                                                                                      #Datastructure(folders)-> is there already a folder for Genre from Array ?
        if not os.path.exists(f"Genres\{genre}"):
            os.makedirs(f"Genres\{genre}")                                                                                                      #if there's already a folder - take this , if not - create new folder
                                                                                                                                                #wenn Ordner -> Datei runterladen und in Ordner speichern
        reqAnswer = requests.get(logo, headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"},stream = True, timeout=10)
        if reqAnswer.status_code != 200:                                                                                                        # entry eventually not found 
            print("download failed", bandName)
                                                                                                                                                # ATTENTION: Sometimes Bandnames cannot be saved as filename cause window doesnt like special chars (e.g. ~ )
                                                                                                                                                # change band name and delete all special chars for the filename
        if bandName in entryArr[0]:
            bandName_new = ''.join(re.findall(r'\w', bandName))
         
        fileName = bandName_new + "_" + logo.split("/")[-1].split("?")[0]                                                                       # filename is build with Bandname and name from Logolink: /284593.jpg?124 -> bla_284593.jpg
        print(fileName)
                                                                                                                                                # Download the file bit by bit to save memory
        with open(f"Genres\{genre}\{fileName}", "wb") as f:
            for chunk in reqAnswer.iter_content(1024):
                f.write(chunk)

## Covert pictures
- to work with all the band logo pictures, we have to convert them in jpgs
- the classifier can't work with the picture format GIF due to higher dimensionality 

In [ ]:
from PIL import Image
import os

# from https://pythonhint.com/post/6698662917296068/pil-convert-gif-frames-to-jpg
# and https://pythonexamples.org/python-check-if-path-is-file-or-directory/

dataPath = 'Genres'

try:
    for subdir in os.listdir(dataPath):
        current_path = os.path.join(dataPath, subdir)
        for file in os.listdir(current_path):
            if os.path.isfile(os.path.join(current_path, file)):                                                                                # exclude folders
                if file[-3:] == "gif":
                    with Image.open(os.path.join(current_path, file)) as im:
                        if im.format == "GIF":
                            for frame_num in range(im.n_frames):
                                                                                                                                                # select the current frame
                                im.seek(frame_num)

                                                                                                                                                # create a new JPEG image with the same size and mode as the GIF frame
                                jpeg_frame = Image.new('RGB', im.size)

                                                                                                                                                # paste the current frame onto the jpeg image
                                jpeg_frame.paste(im)

                                                                                                                                                # save the jpeg image with a filename that includes the current frame number
                                jpeg_filename = str(file).replace(".gif",".jpg")
                                jpeg_frame.save(os.path.join(current_path, jpeg_filename))
                                break
                        else:                                                                                                                   # format is already jpg
                            jpeg_filename = str(file).replace(".gif",".jpg")
                            jpeg_frame.save(os.path.join(current_path, jpeg_filename))
except Exception as e:
    print(e)
